# <font color=blue> DATA CLEANING FOR GOODREADS BOOKS AND REVIEWS SCRAPE </font>

### 1.0 INTRODUCTION

##### This notebook contains code which serves as ```Part II``` of an E-2-E project on the sentiments analysis of Data Science books reviews.

##### The need to do this cleaning owes to the format in which a lot of the features were scraped from the ```Goodreads``` website. The output of the notebook would be a thoroughly cleaned dataset, ready for analysis.

#### 1.1 IMPORT PACKAGES & DEPENDENCIES

In [1]:

import pandas as pd
import numpy as np

import datetime
# from datetime import datetime
import time

import re

from tqdm import tqdm

# import image module
from IPython.display import Image

#### 1.2 PREVIEW THE DATA TO BE CLEANED

In [2]:
df = pd.read_excel('auto_updated_genres.xlsx').dropna(axis=1, how='all')
df = df.copy()
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   title                           1411 non-null   object 
 1   title1                          1411 non-null   object 
 2   author_names                    1411 non-null   object 
 3   genres                          531 non-null    object 
 4   num_pages                       1411 non-null   object 
 5   paperback                       1411 non-null   object 
 6   rating                          1411 non-null   float64
 7   rating1                         1411 non-null   object 
 8   rating_count                    1411 non-null   object 
 9   rating_count1                   1411 non-null   object 
 10  editions                        1411 non-null   object 
 11  publication_info_firstedition   1411 non-null   object 
 12  publication_info_firstedition1  14

None

In [3]:
df.head()

,title,title1,author_names,genres,num_pages,paperback,rating,rating1,rating_count,rating_count1,...,foreward1,price,price1,five_star,four_star,three_star,two_star,one_star,reviews,product_url
0,30-Second Data Science: The 50 Key Principles ...,30-Second Data Science: The 50 Key Principles ...,['Liberty Vittert'],GenresReferenceNonfictionComputers,160 pages,"160 pages, Hardcover",3.68,3.68,34 ratings,34ÃÂ ratings,...,30-Second Data Science is the quickest way to ...,$14.99,"['Kindle $14.99', 'Kindle $14.99']",7 (20%),10 (29%),16 (47%),1 (2%),0 (0%),[('30-Second Data Science is the quickest way ...,https://www.goodreads.com/book/show/51181047-3...
1,A Simple Introduction to Data Science,A Simple Introduction to Data Science,"['Lars Nielsen', 'noreen burlingame']",GenresTechnologyNonfictionBusiness,76 pages,"76 pages, Paperback",3.47,3.47,75 ratings,75ÃÂ ratings,...,Lars Nielsen and Noreen Burlingame provide a b...,$2.99,"['Kindle $2.99', 'Kindle $2.99']",14 (18%),23 (30%),24 (32%),12 (16%),2 (2%),[('Lars Nielsen and Noreen Burlingame provide ...,https://www.goodreads.com/book/show/16286023-a...
2,Adventures in Financial Data Science: The empi...,Adventures in Financial Data Science: The empi...,['Graham Giller'],GenresFinance,Kindle Edition,Kindle Edition,4.00,4,1 rating,1ÃÂ rating,...,NaN,$39.99,"['Kindle $39.99', 'Kindle $39.99']",0 (0%),1 (100%),0 (0%),0 (0%),0 (0%),"[('', '\n'), ('', '\n'), ('', '\n')]",https://www.goodreads.com/book/show/55932069-a...
3,Agile Data Science 2.0: Building Full-Stack Da...,Agile Data Science 2.0: Building Full-Stack Da...,['Russell Jurney'],GenresNonfictionTechnologyComputersComputer Sc...,349 pages,"349 pages, Paperback",3.57,3.57,47 ratings,47ÃÂ ratings,...,Data science teams looking to turn research in...,$38.49,"['Kindle $38.49', 'Kindle $38.49']",8 (17%),20 (42%),10 (21%),9 (19%),0 (0%),[('Data science teams looking to turn research...,https://www.goodreads.com/book/show/30014113-a...
4,AI & Data Literacy: Empowering Citizens of Dat...,AI & Data Literacy: Empowering Citizens of Dat...,['Bill Schmarzo'],GenresArtificial Intelligence,238 pages,Not Found,4.50,Not Found,2 ratings,Not Found,...,Not Found,$9.99,[],1 (33%),2 (66%),0 (0%),0 (0%),0 (0%),"[(""Learn the key skills and capabilities that ...",https://www.goodreads.com/book/show/170997947-...


### 2.0 CLEANING

#### Outlined Steps
***
1. ```title```:        drop the other title column and leave as is.
2. ```author_names```: remove brackets and parenthesis.
3. ```genre```:        separate by each occurence of uppercase letters.
4. ```num_pages```:    extract the number of pages from the text and leave rows without numbers as NULL.
5. ```paperback```:    drop.
6. ```rating```:       drop the other rating column and leave as is.
7. ```rating_count```: extract the rating number and drop the other rating_count column.
8. ```foreward```:     keep as is.
9. ```edition```:      keep as is.
10. ```publication```: extract book format and date of publication details from _publication_info_firstedition1_ into separate columns. drop the other.
11. ```foreward```:    drop _foreward1_ and keep other as is.
12. ```price```:       drop _price1_, change the rows with '_unlimited_' price to 0, and 'Amazon' to NaN.
13. ```star ratings```:on the distribution of the number of stars per review, remove the %.
14. ```reviews```:     drop; it was scraped more comprehensively in another table.
14. ```product_url```: keep as is.

Finally, format each column to appropriate data type
***


#### 2.1 Column-by-Column Extraction/Cleaning

In [4]:
#view current column names
df.columns

Index(['title', 'title1', 'author_names', 'genres', 'num_pages', 'paperback',
       'rating', 'rating1', 'rating_count', 'rating_count1', 'editions',
       'publication_info_firstedition', 'publication_info_firstedition1',
       'foreward', 'foreward1', 'price', 'price1', 'five_star', 'four_star',
       'three_star', 'two_star', 'one_star', 'reviews', 'product_url'],
      dtype='object')

##### 2.1.1 Drop Columns Assigned for Dropping

In [5]:
#drop all columns identified to be dropped
cols_to_drop = ['title1','paperback','rating1','rating_count1','publication_info_firstedition',
                'foreward1','price1','reviews']
df = df.drop(cols_to_drop, axis=1)
df.iloc[10:13]

,title,author_names,genres,num_pages,rating,rating_count,editions,publication_info_firstedition1,foreward,price,five_star,four_star,three_star,two_star,one_star,product_url
10,Be the Outlier: How to Ace Data Science Interv...,['Shrilata Murthy'],NaN,Kindle Edition,3.75,20 ratings,3 editions,"['Kindle Edition', 'Published August 4, 2020']",According to LinkedIn's third annual U.S. Emer...,$5.99,5 (25%),8 (40%),5 (25%),1 (5%),1 (5%),https://www.goodreads.com/book/show/55000016-b...
11,Big Data and Social Science: A Practical Guide...,"['Ian Foster (Editor)', 'Rayid Ghani (Editor)'...",NaN,376 pages,2.67,6 ratings,7 editions,"['376 pages, Hardcover', 'Published August 9, ...",Both Traditional Students and Working Professi...,on,0 (0%),1 (16%),3 (50%),1 (16%),1 (16%),https://www.goodreads.com/book/show/29395434-b...
12,Build a Career in Data Science,"['Emily Robinson (Goodreads Author)', 'Jacque...",GenresComputer ScienceProgrammingTechnologyNon...,250 pages,4.44,148 ratings,1 edition,"['250 pages, Hardcover', 'Published January 1,...",Build a Career in Data Science is your guide t...,$28.99,88 (59%),41 (27%),18 (12%),1 (<1%),1 (<1%),https://www.goodreads.com/book/show/52661559-b...


##### 2.1.2 Separate Author Names

In [6]:
author = []
for authors in df.author_names:
    auth = authors[2:-2].replace("'","").replace("[","").replace("]","").replace('(Editor)','')
    try:
        main_author = auth.split(',', maxsplit=1)[0]#authors[0]
        co_authors = auth.split(',', maxsplit=1)[1]#authors[1:]
    except IndexError:
        co_authors = np.nan
    
    author_data = [main_author, co_authors]
    author.append(author_data)


In [7]:
authors_ = pd.DataFrame(author, columns=['author','co_author(s)'], index=None)

df = pd.concat([df, authors_], axis=1)


##### 2.1.3 Genres

In [8]:
re_gen = []
for g in df['genres']:
    try:
        
        gens = ', '.join([s for s in re.split("([A-Z][^A-Z]*)", g) if s])
        gens = gens.replace('Not , Found', 'Not Found')#[0:]
        re_gen.append(gens)
    except TypeError:
        continue


In [9]:

genres_ = pd.DataFrame(re_gen, index=None, columns=['genre(s)'])
#publication_date_.head()

df = pd.concat([df, genres_], axis=1)


##### 2.1.4 Number of Pages

In [10]:
#Number of pages
page_info = []
for p in df.num_pages:
    if 'pages' in p:
        #print(p.split(' pages')[0])
        pg = p.split(' pages')[0]
    else:
        #print()
        pg = np.NaN
    page_info.append(pg)

In [11]:
page_info_ = pd.Series(page_info)
df.num_pages = page_info_

df.num_pages = df.num_pages.astype('Int32')

##### 2.1.5 Number of Ratings

In [12]:
#Count of ratings received
ratings_info = []
for r in df.rating_count:
    #print(r)
    rt = r.split(' rat')[0].replace(',','')
    ratings_info.append(rt)

In [13]:
ratings_info_ = pd.Series(ratings_info)
df.rating_count = ratings_info_

df.rating_count = df.rating_count.apply(pd.to_numeric)

##### 2.1.6 Editions

In [14]:
#Count of ratings received
editions_info = []
for ed in df.editions:
    #print(r)
    edt = ed.split(' edit')[0].replace(',','')
    editions_info.append(rt)

In [15]:
editions_info_ = pd.Series(editions_info)
df.editions = editions_info_

df.editions = df.editions.apply(pd.to_numeric)

##### 2.1.7 Publication Date & Format

In [16]:
publication = []
for pb in df.publication_info_firstedition1:
    if 'Published' in pb:
        start = pb.find('Published ')
        pbd = pb[start+10:-2]

    else:
        #print('No Publication Date')
        pbd = np.nan

    publication.append(pbd)    


In [17]:
publication_date_ = pd.DataFrame(publication, index=None, columns=['publication_date'])
#publication_date_.head()

df = pd.concat([df, publication_date_], axis=1)


In [18]:
# Define a function to convert a date string to a datetime object
def convert_to_date(date_string):
    
    try:
        date_format = '%B %d, %Y'
        date = pd.to_datetime(date_string, format=date_format)
    except (ValueError,TypeError):
        date_format1 = '%B %d %Y'
        date = pd.to_datetime(date_string, format=date_format1)
    
    return date

# Apply the function to the DataFrame column
df['publication_date'] = df['publication_date'].apply(convert_to_date)

In [19]:
#book publication format

book_format = []
for bk in df.publication_info_firstedition1:
    if ('ebook' in bk or 'CD-ROM' in bk):
        eb = 'ebbok'

    elif 'Leather' in bk:
        eb = 'Leather bound'
    
    elif 'Paperback' in bk:
        eb = 'Paperback'
    
    elif 'Leather' in bk:
        eb = 'Leather bound'
    
    elif 'Kindle' in bk:
        eb = 'Kindle Edition'
        
    elif 'Hardcover' in bk:
        eb = 'Hardcover'
    
    else:
        eb = 'Unknown Binding'

    book_format.append(eb)    

In [20]:
book_format_ = pd.DataFrame(book_format, index=None, columns=['book_format'])
#ebook_check_.head()

df = pd.concat([df, book_format_], axis=1)


##### 2.1.8 Cost of Book (if available for purchase on Goodreads)

In [21]:
book_cost = []
for pr in df.price:
    if '$' in pr:
        prc = pr[1:]

    else:
        prc = np.nan

    book_cost.append(prc) 

In [22]:

book_cost_ = pd.DataFrame(book_cost, index=None, columns=['book_cost($)'])

df = pd.concat([df, book_cost_], axis=1)

df['book_cost($)'] = df['book_cost($)'].astype('float')

##### 2.1.9 Mode of Accessing Book Item

In [23]:
book_access = []
for ac in df.price:
    if '$' in ac:
        acc = 'Purchase on Goodreads'

    elif 'on' in ac:
        acc = 'Purchase on Amazon'
        
    else:
        acc='Kindle Unlimited'

    book_access.append(acc) 

In [24]:
#
book_access_ = pd.DataFrame(book_access, index=None, columns=['book_access'])

df = pd.concat([df, book_access_], axis=1)

##### 2.1.10 Distribution of Ratings Counts

In [25]:
df['five_star'] = [i[0].strip() for i in df['five_star'].str.split('(')]
df['four_star'] = [i[0].strip() for i in df['four_star'].str.split('(')]
df['three_star'] = [i[0].strip() for i in df['three_star'].str.split('(')]
df['two_star'] = [i[0].strip() for i in df['two_star'].str.split('(')]
df['one_star'] = [i[0].strip() for i in df['one_star'].str.split('(')]

In [26]:
#convert to numeric --- int32
star_dist = [i for i in df.columns.to_list() if '_star' in i]
df[star_dist] = df[star_dist].astype('int')

In [27]:
#df[star_dist] = df[['five_star', 'four_star', 'three_star', 'two_star', 'one_star']].astype('int')

##### 2.1.11 Drop More Columns ---

In [28]:
#Drop the Price and Publicationinfo columns from the original file
#Since the needed information has been extracted from these columns

df.drop(['price', 'publication_info_firstedition1', 'author_names', 'genres'], axis=1, inplace=True)

In [29]:
#preview information
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1411 entries, 0 to 1410
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             1411 non-null   object        
 1   num_pages         1085 non-null   Int32         
 2   rating            1411 non-null   float64       
 3   rating_count      1411 non-null   int64         
 4   editions          1411 non-null   int64         
 5   foreward          1366 non-null   object        
 6   five_star         1411 non-null   int32         
 7   four_star         1411 non-null   int32         
 8   three_star        1411 non-null   int32         
 9   two_star          1411 non-null   int32         
 10  one_star          1411 non-null   int32         
 11  product_url       1411 non-null   object        
 12  author            1411 non-null   object        
 13  co_author(s)      338 non-null    object        
 14  genre(s)          531 no

In [35]:
#rename columns & re-arange
df.rename(columns = {'rating':'avg_rating', 'rating_count':'rating_volume', 
                     'genre(s)':'genres', 'book_cost($)':'price'}, inplace = True)

In [37]:
re_cols = ['title', 'author', 'co_author(s)', 'avg_rating', 'rating_volume', 'genres',
           'price', 'num_pages', 'editions','foreward', 'five_star', 'four_star',
           'three_star', 'two_star','one_star', 'publication_date',
           'book_format', 'book_access', 'product_url']

df = df[re_cols]

In [30]:
#save cleaned data to working directory
df.to_csv('cleaned_books.csv', index=False)